# Importing Requried Libraries

In [3]:
import torch 
import os
import numpy as np 
from torchvision.models.vision_transformer import VisionTransformer
from Dataset_Loader import ImageDataset
# from ViT import ViT
from torch import Tensor
from torch import nn,optim
from torch.utils.data import DataLoader,TensorDataset
import torch

from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [5]:
device

'cuda'

# Hyper Parameters

In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32
SHUFFLE = True
EPOCHS = 50
PATCH_SIZE = 16
NUM_LAYERS = 2
NUM_HEADS = 2
HIDDEN_DIM = 128
MLP_DIM = 256
DROPOUT = 0.2
LEARNING_RATE = 1e-4
TRAIN_PATH = "Dataset/train/"
VAL_PATH = "Dataset/val1/"

## Creating the DataLoader

In [7]:
train_image_dataset = ImageDataset(path=TRAIN_PATH,img_size=IMG_SIZE,batch_size=BATCH_SIZE,shuffle=SHUFFLE)
val_image_dataset =  ImageDataset(path=VAL_PATH,img_size=IMG_SIZE,batch_size=BATCH_SIZE,shuffle=SHUFFLE)

In [8]:
train_loader = train_image_dataset.get_dataloader()
val_loader = val_image_dataset.get_dataloader()

In [9]:
train_image_dataset.get_labels()

{'Safe': 0, 'notSafe': 1}

# Model Training:

### Creating Model

In [10]:
vit_model = VisionTransformer(
    image_size = IMG_SIZE,
    patch_size = PATCH_SIZE,
    num_layers = NUM_LAYERS,
    num_heads = NUM_HEADS,
    hidden_dim = HIDDEN_DIM,
    mlp_dim = MLP_DIM,
    dropout = DROPOUT,
)
seq_vit_model = torch.nn.Sequential(
    vit_model,
    nn.Linear(1000,2),
    nn.Sigmoid()
)
seq_vit_model.to(device)
seq_vit_model

Sequential(
  (0): VisionTransformer(
    (conv_proj): Conv2d(3, 128, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.2, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (dropout): Dropout(p=0.2, inplace=False)
          (ln_2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=128, out_features=256, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.2, inplace=False)
            (3): Linear(in_features=256, out_features=128, bias=True)
            (4): Dropout(p=0.2, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): LayerNorm(

### Defining Loss Functions

In [11]:
LOSS_FUNCTION = torch.nn.CrossEntropyLoss()
OPTIMIZER = torch.optim.Adam(seq_vit_model.parameters(), lr = LEARNING_RATE)

# Model Training

In [25]:
 # (batch_size, channels, height, width).

In [12]:
history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

for epoch in range(EPOCHS):
    # Training phase
    seq_vit_model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        images, labels = images.to(device), labels.to(device)

        # Ensure correct shape for Vision Transformer
        if images.shape[1] != 3:  # Expecting (batch, 3, height, width)
            images = images.permute(0, 3, 1, 2)  # Convert NHWC → NCHW
        
        OPTIMIZER.zero_grad()  # Clear gradients
        outputs = seq_vit_model(images)  # Forward pass

        # Compute loss
        loss = LOSS_FUNCTION(outputs, labels)
        loss.backward()  # Backpropagation
        OPTIMIZER.step()  # Update weights

        # Update training metrics
        train_loss += loss.item()
        correct_train += (outputs.argmax(dim=1) == labels).sum().item()  # Remove labels.argmax(dim=1)
        total_train += labels.size(0)

    train_loss /= len(train_loader)
    train_acc = correct_train / total_train
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)

    # --------------------> Validation phase <------------------------
    seq_vit_model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            images, labels = images.to(device), labels.to(device)

            # Ensure correct shape
            if images.shape[1] != 3:
                images = images.permute(0, 3, 1, 2)

            outputs = seq_vit_model(images)

            # Compute loss
            loss = LOSS_FUNCTION(outputs, labels)
            val_loss += loss.item()

            # Update validation metrics
            correct_val += (outputs.argmax(dim=1) == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= len(val_loader)
    val_acc = correct_val / total_val
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    # Print epoch summary
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


Epoch 1/50 [Val]: 100%|██████████████████████████████████████████████████████████████| 195/195 [01:24<00:00,  2.30it/s]


Epoch 1/50, Train Loss: 0.5642, Train Acc: 0.7457, Val Loss: 0.5222, Val Acc: 0.7836


Epoch 2/50 [Val]: 100%|██████████████████████████████████████████████████████████████| 195/195 [01:18<00:00,  2.49it/s]


Epoch 2/50, Train Loss: 0.5246, Train Acc: 0.7814, Val Loss: 0.5143, Val Acc: 0.7923


Epoch 3/50 [Val]: 100%|██████████████████████████████████████████████████████████████| 195/195 [01:08<00:00,  2.84it/s]


...

Epoch 48/50, Train Loss: 0.4322, Train Acc: 0.8764, Val Loss: 0.5101, Val Acc: 0.7994


Epoch 49/50 [Val]: 100%|█████████████████████████████████████████████████████████████| 195/195 [01:07<00:00,  2.91it/s]


Epoch 49/50, Train Loss: 0.4302, Train Acc: 0.8789, Val Loss: 0.5336, Val Acc: 0.7743


Epoch 50/50 [Val]: 100%|█████████████████████████████████████████████████████████████| 195/195 [01:05<00:00,  2.98it/s]

Epoch 50/50, Train Loss: 0.4350, Train Acc: 0.8741, Val Loss: 0.5033, Val Acc: 0.8045


In [16]:
max(history["train_acc"]),max(history["val_acc"])

(0.8805373420758036, 0.8186786690242727)

In [19]:
import pandas as pd
model_history = pd.DataFrame(history)
model_history.to_csv("Conv + Transformers.csv")

In [20]:
torch.save(seq_vit_model,"Saved Models/Conv + Transformers.pth")